# Transformer Encoder Decoder from scratch

Este cuaderno está basado en:

- https://blog.floydhub.com/the-transformer-in-pytorch/
- https://arxiv.org/pdf/1706.03762.pdf
- https://txt.cohere.com/what-are-transformer-models/
- https://jalammar.github.io/illustrated-transformer/


# Fundamentos de los bloques de la arquitectura Transformers

Veremos el modelo encoder-decoder del artículo:

[Attention is All You Need](https://arxiv.org/pdf/1706.03762.pdf).

Utilizaremos la biblioteca [PyTorch]() para poder entender cómo funciona el entrenamiento de estos modelos. Este cuaderno se podría utilizar para entrenar un GPT, pero llevaría días.

Lo haremos en un subconjunto de los datos para ver cómo funciona.


## La arquitectura

![imagen](https://i.imgur.com/YPjbqW6.png)


In [ ]:
import torch
import torch.nn as nn
import math

## Embedding

![imagen](https://i.imgur.com/sFlEZ2e.png)

El primer paso será transformar nuestra secuencia de tokens en vectores de incorporación para capturar información semántica. Esta capa de incorporación mejorará durante el entrenamiento del modelo, proporcionando una representación vectorial rica de los tokens.

In [ ]:
# Definimos una clase para la capa de embedding de entrada que hereda de nn.Module
class InputEmbeddings(nn.Module):
    def __init__(self, d_model: int, vocab_size: int) -> None:
        """
        Este constructor inicializa la capa de embedding.

        Parámetros:
        vocab_size - el tamaño de nuestro vocabulario, es decir, el número total de palabras únicas que podemos procesar.
        d_model - la dimensión de nuestros embeddings y la dimensión de entrada para nuestro modelo, que también será el tamaño de las salidas de la capa de embedding.
        """
        # Inicializamos la clase base.
        super().__init__()

        # Guardamos el tamaño del vocabulario y la dimensión del modelo como atributos de la clase.
        self.vocab_size = vocab_size
        self.d_model = d_model

        # Creamos una capa de embedding, que convertirá índices de palabras en vectores de alta dimensión.
        self.embedding = nn.Embedding(vocab_size, d_model)

    def forward(self, x):
        # Este método procesa las entradas a través de la capa de embedding.
        # Multiplica las embeddings por la raíz cuadrada de la dimensión del modelo para normalizar la magnitud de los vectores de embedding.
        return self.embedding(x) * math.sqrt(self.d_model)


In [ ]:
vocab_size = 1000  # Tamaño del vocabulario: número total de palabras únicas
d_model = 512  # Dimensión de los embeddings y la dimensión de entrada para el modelo
batch_size = 16  # Tamaño del lote: número de ejemplos procesados juntos
sequence_length = 350  # Longitud de la secuencia: número de tokens por entrada

embedding_layer = InputEmbeddings(d_model, vocab_size)

dummy_input = torch.randint(0, vocab_size, (batch_size, sequence_length))

output = embedding_layer(dummy_input)

print(f"Forma del output: {output.shape}")  # Imprime la forma del output


Forma del output: torch.Size([16, 350, 512])


## Positional Encoding

![imagen](https://i.imgur.com/IIA3NK3.png)

Para indicar la posición de cada token en la secuencia sin usar recurrencia ni convoluciones, inyectaremos información posicional en las incrustaciones de entrada, siguiendo el método propuesto en el artículo, mediante una combinación específica de funciones.


In [ ]:
# Definimos la clase PositionalEncoding, que hereda de nn.Module (un módulo estándar de PyTorch)
class PositionalEncoding(nn.Module):
  # El constructor de la clase recibe dimensiones del modelo (d_model), longitud de la secuencia (seq_len),
  # y la tasa de dropout (dropout) como argumentos.

  def __init__(self, d_model: int, seq_len: int, dropout: float) -> None:
    super().__init__()  # Inicializamos la superclase nn.Module
    self.d_model = d_model  # Dimensión del modelo
    self.seq_len = seq_len  # Longitud de la secuencia de entrada
    self.dropout = nn.Dropout(dropout)  # Definimos una capa de dropout para evitar el sobreajuste

    # Inicializamos una matriz de ceros para los embeddings posicionales
    positional_embeddings = torch.zeros(seq_len, d_model)
    # Creamos un vector de secuencia posicional usando arange, que representa el índice posicional de cada token
    positional_sequence_vector = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1)
    # Calculamos los valores del modelo posicional, que se utilizan para generar las funciones seno y coseno
    positional_model_vector = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
    # Asignamos las funciones seno a los embeddings posicionales en las posiciones pares
    positional_embeddings[:, 0::2] = torch.sin(positional_sequence_vector * positional_model_vector)
    # Asignamos las funciones coseno a los embeddings posicionales en las posiciones impares
    positional_embeddings[:, 1::2] = torch.cos(positional_sequence_vector * positional_model_vector)
    # Añadimos una dimensión extra al principio para poder sumar este tensor con los embeddings de tokens
    positional_embeddings = positional_embeddings.unsqueeze(0)

    # Registramos los embeddings posicionales como un buffer que no se considerará un parámetro del modelo
    # y por lo tanto no se actualizará durante el entrenamiento
    self.register_buffer('positional_embeddings', positional_embeddings)

  # La función forward define cómo se procesa la entrada x a través del módulo
  def forward(self, x):
    # Sumamos los embeddings posicionales a los embeddings de tokens (x), asegurándonos que no se calculan
    # gradientes para los embeddings posicionales durante el entrenamiento
    x = x + (self.positional_embeddings[:, :x.shape[1], :]).requires_grad_(False)
    # Aplicamos la capa de dropout a los embeddings resultantes y retornamos el resultado
    return self.dropout(x)


In [ ]:
# Definimos los parámetros de nuestra codificación posicional
d_model = 512
seq_len = 350
dropout = 0.1

pos_enc = PositionalEncoding(d_model, seq_len, dropout)

batch_size = 16
dummy_input = torch.rand(batch_size, seq_len, d_model)

output = pos_enc(dummy_input)

print(f"Forma del output: {output.shape}")

Forma del output: torch.Size([16, 350, 512])


## Add & Norm


![image](https://i.imgur.com/otdEq4D.png)

### Layer Normalization

El primer paso es añadir la normalización de capas. ¡Puedes leer más sobre esto [aquí!](https://paperswithcode.com/method/layer-normalization)!

La idea básica es que hace que el entrenamiento del modelo sea un poco más fácil y permite al modelo generalizar un poco mejor.

In [ ]:
class LayerNormalization(nn.Module):

    def __init__(self, features: int, epsilon: float = 1e-6) -> None:
        super().__init__()
        self.epsilon = epsilon  # Pequeño valor añadido para estabilidad numérica durante la división
        # Parámetros entrenables que escalan (gamma) y desplazan (beta) la normalización
        self.gamma = nn.Parameter(torch.ones(features))  # Gamma: Parámetro de escala inicializado a 1
        self.beta = nn.Parameter(torch.zeros(features))  # Beta: Parámetro de desplazamiento inicializado a 0

    def forward(self, x):
        # Calcula la media de cada muestra de manera independiente
        mean = x.mean(dim=-1, keepdim=True)
        # Calcula la desviación estándar de cada muestra
        standard_deviation = x.std(dim=-1, keepdim=True)
        # Aplica la normalización de capa y luego escala y desplaza los resultados
        return self.gamma * (x - mean) / (standard_deviation + self.epsilon) + self.beta

Epsilon nos permite evitar divisiones por cero

### Residual Connection

Otra técnica que facilita el entrenamiento del modelo es añadir una conexión residual a las salidas del Bloque de Atención - esto ayuda a prevenir la desvanecimiento del gradiente.

In [ ]:
class ResidualConnection(nn.Module):
  def __init__(self, features: int, dropout: float = 0.1) -> None:
    super().__init__()
    # Creamos una capa de dropout para regularizar el aprendizaje.
    self.dropout = nn.Dropout(dropout)
    # Inicializamos la normalización de capa con el número de características dado.
    self.layernorm = LayerNormalization(features)

  # La función forward define el paso hacia adelante del módulo.
  def forward(self, x, sublayer):
    # Devolvemos la suma de la entrada original x y la salida de la subcapa,
    # aplicando dropout y normalización de capa a la entrada antes de pasarla a la subcapa.
    return x + self.dropout(sublayer(self.layernorm(x)))

## Feed Forward Network

![image](https://i.imgur.com/woEqBjQ.png)

Las redes feed forward tienen dos propósitos en nuestro modelo:

1. Transforman las salidas de atención en un formato que funciona con el bloque siguiente.

2. Ayudan a añadir complejidad para evitar que cada bloque de atención actúe de manera similar.

In [ ]:
class FeedForwardBlock(nn.Module):
  def __init__(self, d_model: int, d_ff: int = 2048, dropout: float = 0.1) -> None:
    """
    Inicializa una capa feed-forward en el modelo de Transformer.

    Argumentos:
    d_model (int): La dimensión del modelo, es decir, el tamaño de los embeddings o vectores de tokens.
    d_ff (int): La dimensión de la red feed-forward interna, usualmente mucho mayor que d_model.
    dropout (float): La tasa de dropout, una medida de regularización que desactiva aleatoriamente
                     partes de las activaciones (nodos) durante el entrenamiento para prevenir el sobreajuste.
    """
    super().__init__()
    # La primera capa lineal transforma los vectores de entrada a una dimensión mayor (d_ff).
    self.linear_1 = nn.Linear(d_model, d_ff)

    # Dropout para añadir regularización y prevenir el sobreajuste.
    self.dropout = nn.Dropout(dropout)

    # La segunda capa lineal transforma los vectores de la dimensión d_ff de nuevo a d_model.
    self.linear_2 = nn.Linear(d_ff, d_model)

  def forward(self, x):
    # Aplica la primera transformación lineal seguida de una función de activación ReLU,
    # luego aplica dropout y finalmente una segunda transformación lineal.
    # La función de activación ReLU se utiliza para añadir no linealidad al modelo.
    return self.linear_2(self.dropout(torch.relu(self.linear_1(x))))

## Multi-Head Attention

![image](https://i.imgur.com/4qOT46y.png)

El nucleo del transformer!

### Multi-Head Attention Class



In [ ]:
class MultiHeadAttention(nn.Module):
  def __init__(self, d_model: int = 512, num_heads: int = 8, dropout: float = 0.1) -> None:
    super().__init__()

    # d_model es la dimensión de los embeddings/representaciones del modelo.
    self.d_model = d_model

    # num_heads es el número de cabezas de atención.
    self.num_heads = num_heads

    # Verifica que d_model sea divisible por el número de cabezas para asegurar una división igual de los vectores.
    assert d_model % num_heads == 0, "d_model no es divisible por num_heads"

    # d_k representa la dimensión de las claves, valores y consultas para cada cabeza de atención.
    self.d_k = d_model // num_heads

    # Define las matrices de pesos para las consultas, claves y valores.
    self.w_q = nn.Linear(d_model, d_model, bias=False)  # Pesos para las consultas
    self.w_k = nn.Linear(d_model, d_model, bias=False)  # Pesos para las claves
    self.w_v = nn.Linear(d_model, d_model, bias=False)  # Pesos para los valores

    # Después de concatenar las salidas de las cabezas, esta matriz de pesos proyecta de nuevo al d_model original.
    self.w_o = nn.Linear(d_model, d_model, bias=False)

    # Define una capa de dropout para regularizar el aprendizaje.
    self.dropout = nn.Dropout(dropout)


### Scaled Dot-Product Attention

![image](https://i.imgur.com/Yp48DuB.png)

Entradas Q, K, V: La atención toma tres conjuntos de entradas; las consultas (Q), las claves (K) y los valores (V). Estas son proyecciones lineales (transformaciones afines) de la entrada en diferentes espacios.

- **MatMul entre Q y K**: Se realiza un *scaled dot* (multiplicación matricial) entre las consultas y las claves. Esto resulta en una matriz de puntuaciones de atención que indica cuánta atención se debe prestar a otros elementos en la secuencia al procesar un elemento específico.

- **Escala**: Las puntuaciones de atención se escalan dividiéndolas por la raíz cuadrada de la dimensión de las claves (sqrt(d_k)). Esto es para evitar gradientes muy pequeños, ya que el *scaled dot* puede aumentar significativamente con el aumento de la dimensión de las claves.

- **Máscara (opcional)**: Si se utiliza una máscara (por ejemplo, para evitar que se preste atención a los tokens de relleno o para implementar la atención causal donde cada posición solo puede atender a posiciones anteriores en la secuencia), esta se aplica antes del softmax. Los valores de la máscara suelen ser 0 (para posiciones que deben considerarse) y -inf o un número muy grande negativo (para posiciones que deben ignorarse) de tal manera que después de aplicar softmax, estas últimas posiciones reciban efectivamente una puntuación de atención de cero.

- **SoftMax**: Se aplica la función softmax a las puntuaciones de atención escaladas para cada posición, resultando en una distribución de probabilidad que suma 1. Esto garantiza que solo se preste atención a las posiciones más relevantes.

- **MatMul con V**: Las puntuaciones de atención softmax se utilizan para ponderar los valores (V). Esto se hace multiplicando las puntuaciones de atención softmax con los valores. Esto significa que cada valor se pondera por cuánta atención la clave correspondiente debería recibir.

- **Salida**: El resultado de este *scaled dot* es el resultado de la atención, que luego se puede pasar a través de más capas en una red o utilizarse para producir una salida.

Este mecanismo de atención permite que el modelo se concentre selectivamente en partes relevantes de la entrada y es fundamental para permitir que los modelos de transformadores manejen secuencias largas y realicen tareas como la traducción automática, el procesamiento del lenguaje natural y más.


In [ ]:
def attention(query, key, value, mask, d_k, dropout: nn.Dropout = None):
  # Calculemos las puntuaciones de atención realizando el producto escalar de los queries y keys,
  # y luego dividiendo por la raíz cuadrada de la dimensión d_k para estabilizar los gradientes.
  attention_scores = (query @ key.transpose(-2, -1)) / math.sqrt(d_k)

  # Si se proporciona una máscara, la aplicamos a las puntuaciones de atención.
  # Esto puede ser útil para evitar que el modelo considere ciertas posiciones (por ejemplo, en los tokens rellenados o en los tokens futuros en el decodificador).
  if mask is not None:
    attention_scores = attention_scores.masked_fill_(mask == 0, -1e9)

  # Normalizamos las puntuaciones de atención usando softmax, para que sumen 1 y se puedan interpretar como probabilidades.
  attention_scores = attention_scores.softmax(dim=-1)

  # Si se ha proporcionado una capa de dropout, la aplicamos a las puntuaciones de atención.
  # Esto ayuda a prevenir el sobreajuste durante el entrenamiento.
  if dropout is not None:
    attention_scores = dropout(attention_scores)

  # Retornamos la salida de la atención, que es el producto escalar de las puntuaciones de atención y los values,
  # junto con las puntuaciones de atención para posibles usos posteriores (como visualizaciones).
  return (attention_scores @ value), attention_scores

### Forward Method

In [ ]:
def forward(self, query, key, value, mask):
  # Aplica una capa lineal a las queries, keys y values para transformarlas en un nuevo espacio.
  query = self.w_q(query)
  key = self.w_k(key)
  value = self.w_v(value)

  # Reorganiza las queries, keys y values para agruparlas según los multi-heads y
  # prepararlas para la operación de atención, separando las dimensiones de cabezales y características.
  query = query.view(query.shape[0], query.shape[1], self.num_heads, self.d_k).transpose(1, 2)
  key = key.view(key.shape[0], key.shape[1], self.num_heads, self.d_k).transpose(1, 2)
  value = value.view(value.shape[0], value.shape[1], self.num_heads, self.d_k).transpose(1, 2)

  # Calcula la atención multi-cabeza utilizando las queries, keys y values modificadas, así como la máscara
  # proporcionada, y aplica un dropout si es necesario.
  x, self.attention_scores = MultiHeadAttention.attention(query, key, value, mask, self.dropout)

  # Reorganiza la salida de la atención para convertirla de nuevo en la forma original
  # (tamaño del lote, longitud de secuencia, características).
  x = x.transpose(1, 2).contiguous().view(x.shape[0], -1, self.num_heads * self.d_k)

  # Aplica otra capa lineal para transformar la salida de la atención multi-cabeza en el espacio deseado.
  return self.w_o(x)

### MultiHeadAttention

In [ ]:
class MultiHeadAttention(nn.Module):
  def __init__(self, d_model: int = 512, num_heads: int = 8, dropout: float = 0.1) -> None:
    super().__init__()
    self.d_model = d_model
    self.num_heads = num_heads
    assert d_model % num_heads == 0, "d_model is not divisible by h"

    self.d_k = d_model // num_heads

    self.w_q = nn.Linear(d_model, d_model, bias=False)
    self.w_k = nn.Linear(d_model, d_model, bias=False)
    self.w_v = nn.Linear(d_model, d_model, bias=False)

    self.w_o = nn.Linear(d_model, d_model, bias=False)

    self.dropout = nn.Dropout(dropout)

  @staticmethod
  def attention(query, key, value, mask, dropout: nn.Dropout = None):
    d_k = query.shape[-1]

    attention_scores = (query @ key.transpose(-2, -1)) / math.sqrt(d_k)

    if mask is not None:
      attention_scores.masked_fill_(mask == 0, -1e9)

    attention_scores = attention_scores.softmax(dim=-1)

    if dropout is not None:
      attention_scores = dropout(attention_scores)

    return (attention_scores @ value), attention_scores

  def forward(self, query, key, value, mask):
    query = self.w_q(query)
    key = self.w_k(key)
    value = self.w_v(value)

    query = query.view(query.shape[0], query.shape[1], self.num_heads, self.d_k).transpose(1, 2)
    key = key.view(key.shape[0], key.shape[1], self.num_heads, self.d_k).transpose(1, 2)
    value = value.view(value.shape[0], value.shape[1], self.num_heads, self.d_k).transpose(1, 2)

    x, self.attention_scores = MultiHeadAttention.attention(query, key, value, mask, self.dropout)

    x = x.transpose(1, 2).contiguous().view(x.shape[0], -1, self.num_heads * self.d_k)

    return self.w_o(x)

## Encoder

Con todos los bloques implementados, vamos a unirlos para crear el Encoder



### Encoder Block

![image](https://i.imgur.com/nwNYZAT.png)


El codificador toma la frase en el idioma fuente (por ejemplo, inglés). Cada palabra se convierte en una representación vectorial utilizando una capa de incrustación (embedding). Luego, un codificador posicional añade información sobre la posición de cada palabra. Esto pasa por múltiples capas de autoatención, donde cada vector de palabra presta atención a todos los otros vectores de palabras para construir representaciones contextuales.


In [ ]:
class EncoderBlock(nn.Module):
  def __init__(self, features: int, self_attention_block: MultiHeadAttention, feed_forward_block: FeedForwardBlock, dropout: float) -> None:
    super().__init__()

    # Asigna el bloque de autoatención MultiHeadAttention a esta clase.
    self.self_attention_block = self_attention_block

    # Asigna el bloque FeedForwardBlock a esta clase.
    self.feed_forward_block = feed_forward_block

    # Crea una lista de conexiones residuales, con dos elementos para este bloque de codificador.
    # Estas son utilizadas para añadir la entrada original (residual) a la salida de cada subcapa,
    # seguido de una normalización de capa.
    self.residual_connections = nn.ModuleList([ResidualConnection(features, dropout) for _ in range(2)])

  def forward(self, x, input_mask):
    # Aplica la primera conexión residual y el bloque de autoatención.
    # La función lambda permite pasar la salida de la autoatención como segundo argumento a la conexión residual.
    x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, input_mask))

    # Aplica la segunda conexión residual y el bloque de alimentación adelante.
    x = self.residual_connections[1](x, self.feed_forward_block)

    # Retorna la salida procesada del bloque de codificador.
    return x

### Encoder Stack


Siguiendo el artículo original - organizaremos estos bloques en un conjunto de 6.

Estos 6 Bloques Codificadores (cada uno con 8 Cabezas de Atención) conformarán nuestro Stack de Codificación.

In [ ]:
class EncoderStack(nn.Module):
  def __init__(self, features: int, layers: nn.ModuleList) -> None:
    super().__init__()
    self.layers = layers
    self.norm = LayerNormalization(features) # Inicializamos la normalización de capa que se aplicará al final

  def forward(self, x, mask):
    for layer in self.layers: # Iteramos sobre todas las capas del encoder
      x = layer(x, mask) # Aplicamos cada capa a la entrada x con la máscara proporcionada
    return self.norm(x) # Retornamos la salida normalizada de la última capa


## Decoder

Vamos a hacer lo mismo con el Decoder!

### Decoder Block

![image](https://i.imgur.com/HtAAXZc.png)



El decodificador repite la frase en el idioma objetivo (por ejemplo, español). También convierte las palabras en vectores y añade información posicional. Luego pasa por capas de autoatención. Aquí, se aplica una máscara de manera que cada palabra solo puede ver las palabras anteriores, no las posteriores.

El decodificador también realiza atención sobre la salida del encoder. Esto permite que cada palabra en francés encuentre conexiones relevantes con las palabras en inglés.

In [ ]:
class DecoderBlock(nn.Module):
  def __init__(self, features: int, self_attention_block: MultiHeadAttention, cross_attention_block: MultiHeadAttention, feed_forward_block: FeedForwardBlock, dropout: float) -> None:
    super().__init__()
    self.self_attention_block = self_attention_block
    self.cross_attention_block = cross_attention_block
    self.feed_forward_block = feed_forward_block
    self.residual_connections = nn.ModuleList([ResidualConnection(features, dropout) for _ in range(3)])

  def forward(self, x, encoder_output, input_mask, target_mask):
    # Aplicación de la primera conexión residual y el bloque de atención propia
    x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, target_mask))
    # Aplicación de la segunda conexión residual y el bloque de atención cruzada con la salida del codificador
    x = self.residual_connections[1](x, lambda x: self.cross_attention_block(x, encoder_output, encoder_output, input_mask))
    # Aplicación de la tercera conexión residual y el bloque de avance rápido
    x = self.residual_connections[2](x, self.feed_forward_block)
    return x


### Decoder Stack

Usaremos el mismo número de Bloques Decodificadores que usamos en los Bloques Codificadores - lo que nos deja con 6 Bloques Decodificadores en nuestra Pila de Decodificadores.

In [ ]:
class DecoderStack(nn.Module):
  def __init__(self, features: int, layers: nn.ModuleList) -> None:
    super().__init__()
    self.layers = layers
    self.norm = LayerNormalization(features)

  # Definimos la función forward que será llamada durante la propagación hacia adelante
  def forward(self, x, encoder_output, input_mask, target_mask):
    # Iteramos a través de cada capa en la lista de capas
    for layer in self.layers:
      x = layer(x, encoder_output, input_mask, target_mask)  # Pasamos las entradas a través de la capa
    return self.norm(x)


## Linear Projection Layer

Después de las capas de autoatención del decodificador y de atención entre el codificador y el decodificador, tenemos un vector de contexto que representa cada palabra española. Este vector de contexto tiene una alta dimensión (por ejemplo, 512 o 1024).

Tomamos este vector de contexto y generamos una distribución de probabilidad sobre el vocabulario francés para poder elegir la próxima palabra traducida.

La capa de proyección lineal ayuda con esto. Proyecta el vector de contexto en un vector mucho más grande llamado la distribución del vocabulario - una entrada por cada palabra del vocabulario.

Por ejemplo, si nuestro vocabulario catalán tiene 50,000 palabras, la distribución del vocabulario tendrá 50,000 dimensiones. Cada dimensión corresponde a la probabilidad de que aquella palabra catalana sea la traducción correcta.

In [ ]:
class LinearProjectionLayer(nn.Module):
  def __init__(self, d_model, vocab_size) -> None:
    super().__init__()
    self.proj = nn.Linear(d_model, vocab_size)

  def forward(self, x) -> None:
    return self.proj(x)

## The Transformer

Si juntamos todas las partes... tenemos el **TRANSFORMER**!


In [ ]:
class Transformer(nn.Module):
  def __init__(self, encoder: EncoderBlock, decoder: DecoderBlock, src_embed: InputEmbeddings, tgt_embed: InputEmbeddings, src_pos: PositionalEncoding, tgt_pos: PositionalEncoding, projection_layer: LinearProjectionLayer) -> None:
    super().__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.src_embed = src_embed
    self.tgt_embed = tgt_embed
    self.src_pos = src_pos
    self.tgt_pos = tgt_pos
    self.projection_layer = projection_layer

  def encode(self, src, src_mask):
    src = self.src_embed(src)
    src = self.src_pos(src)
    return self.encoder(src, src_mask)

  def decode(self, encoder_output: torch.Tensor, src_mask: torch.Tensor, tgt: torch.Tensor, tgt_mask: torch.Tensor):
    tgt = self.tgt_embed(tgt)
    tgt = self.tgt_pos(tgt)
    return self.decoder(tgt, encoder_output, src_mask, tgt_mask)

  def project(self, x):
    return self.projection_layer(x)

## Construyendo Nuestro Transformer

Utilizaremos esta función de ayuda para crear nuestro Transformer...


In [ ]:
def build_transformer(input_vocab_size: int, target_vocab_size: int, input_seq_len: int, target_seq_len: int, d_model: int=512, N: int=6, num_heads: int=8, dropout: float=0.1, d_ff: int=2048) -> Transformer:
  # Creación de las incrustaciones de entrada para el vocabulario de entrada
  input_embeddings = InputEmbeddings(d_model, input_vocab_size)
  # Creación de las incrustaciones de entrada para el vocabulario objetivo
  target_embeddings = InputEmbeddings(d_model, target_vocab_size)

  # Codificación posicional para la secuencia de entrada
  input_position = PositionalEncoding(d_model, input_seq_len, dropout)
  # Codificación posicional para la secuencia objetivo
  target_position = PositionalEncoding(d_model, target_seq_len, dropout)

  # Inicialización de la lista de bloques del codificador
  encoder_blocks = []

  # Construcción de los bloques del codificador, repitiendo N veces
  for _ in range(N):
    encoder_self_attention_block = MultiHeadAttention(d_model, num_heads, dropout)  # Bloque de atención para el codificador
    feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)  # Bloque de avance rápido
    encoder_block = EncoderBlock(d_model, encoder_self_attention_block, feed_forward_block, dropout)
    encoder_blocks.append(encoder_block)  # Añadir el bloque completo al codificador

  # Inicialización de la lista de bloques del decodificador
  decoder_blocks = []

  # Construcción de los bloques del decodificador, repitiendo N veces
  for _ in range(N):
    decoder_self_attention_block = MultiHeadAttention(d_model, num_heads, dropout)  # Bloque de atención propia del decodificador
    decoder_cross_attention_block = MultiHeadAttention(d_model, num_heads, dropout)  # Bloque de atención cruzada para el decodificador
    feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)  # Bloque de avance rápido
    decoder_block = DecoderBlock(d_model, decoder_self_attention_block, decoder_cross_attention_block, feed_forward_block, dropout)
    decoder_blocks.append(decoder_block)  # Añadir el bloque completo al decodificador

  # Empaquetado de todos los bloques del codificador en una pila
  encoder_stack = EncoderStack(d_model, nn.ModuleList(encoder_blocks))
  # Empaquetado de todos los bloques del decodificador en una pila
  decoder_stack = DecoderStack(d_model, nn.ModuleList(decoder_blocks))

  # Capa de proyección lineal que mapea las salidas del decodificador al tamaño del vocabulario objetivo
  linear_projection_layer = LinearProjectionLayer(d_model, target_vocab_size)

  # Construcción del transformador con todas las partes definidas anteriormente
  transformer = Transformer(encoder_stack, decoder_stack, input_embeddings, target_embeddings, input_position, target_position, linear_projection_layer)

  # Inicialización de los parámetros del transformador usando Xavier Uniform
  for p in transformer.parameters():
    if p.dim() > 1:
      nn.init.xavier_uniform_(p)

  return transformer


# ¡Vamos a entrenar nuestro TRANSFORMER!

- Utilizaremos este recurso: [repositorio](https://github.com/hkproj/pytorch-transformer/tree/main)

Usaremos su dataset de PyTorch para traducción de inglés a francés.


## Creación de Conjuntos de Datos

El `BilingualDataset` es una herramienta específica de PyTorch diseñada para facilitar el entrenamiento de modelos de traducción automática. Utiliza tokenizadores específicos para cada idioma para convertir pares de frases en formato de texto a una forma numérica que los modelos de machine learning pueden procesar. Este proceso incluye ajustar la longitud de todas las frases a un máximo predefinido, agregando tokens especiales que indican el inicio y el final de las frases, así como tokens de relleno para asegurar que todas las entradas y salidas tengan una longitud uniforme.

Cuando se solicita un ejemplo de este conjunto de datos, se realiza la tokenización de las frases en los idiomas fuente y objetivo, se ajustan sus longitudes, y se generan tres tipos de tensores necesarios para el modelo: la entrada del codificador, la entrada del decodificador, y las etiquetas de salida. También se elaboran máscaras específicas que ayudan al modelo a distinguir entre la información real y los tokens de relleno, y a asegurar que la predicción de cada nuevo token por el decodificador se base únicamente en información previa, evitando cualquier influencia de tokens futuros.

Esta configuración prepara el terreno para un entrenamiento eficaz de modelos secuencia-a-secuencia, optimizando el proceso de aprendizaje al tratar la naturaleza inherentemente secuencial del lenguaje, donde la predicción de palabras depende de las precedentes y no de las que vienen después.


In [ ]:
from torch.utils.data import Dataset

class BilingualDataset(Dataset):
  def __init__(self, ds, tokenizer_src, tokenizer_tgt, src_lang, tgt_lang, seq_len):
    super().__init__()
    self.seq_len = seq_len
    self.ds = ds
    self.tokenizer_src = tokenizer_src
    self.tokenizer_tgt = tokenizer_tgt
    self.src_lang = src_lang
    self.tgt_lang = tgt_lang

    self.sos_token = torch.tensor([tokenizer_tgt.token_to_id("[SOS]")], dtype=torch.int64)
    self.eos_token = torch.tensor([tokenizer_tgt.token_to_id("[EOS]")], dtype=torch.int64)
    self.pad_token = torch.tensor([tokenizer_tgt.token_to_id("[PAD]")], dtype=torch.int64)

  def __len__(self):
    return len(self.ds)

  def __getitem__(self, idx):
    src_target_pair = self.ds[idx]
    src_text = src_target_pair['translation'][self.src_lang]
    tgt_text = src_target_pair['translation'][self.tgt_lang]

    enc_input_tokens = self.tokenizer_src.encode(src_text).ids[:self.seq_len - 2]
    dec_input_tokens = self.tokenizer_tgt.encode(tgt_text).ids[:self.seq_len - 1]

    enc_num_padding_tokens = self.seq_len - len(enc_input_tokens) - 2
    dec_num_padding_tokens = self.seq_len - len(dec_input_tokens) - 1

    encoder_input = torch.cat(
        [
            self.sos_token,
            torch.tensor(enc_input_tokens, dtype=torch.int64),
            self.eos_token,
            torch.tensor([self.pad_token] * enc_num_padding_tokens, dtype=torch.int64),
        ],
        dim=0,
    )

    decoder_input = torch.cat(
        [
            self.sos_token,
            torch.tensor(dec_input_tokens, dtype=torch.int64),
            torch.tensor([self.pad_token] * dec_num_padding_tokens, dtype=torch.int64),
        ],
        dim=0,
    )

    label = torch.cat(
        [
            torch.tensor(dec_input_tokens, dtype=torch.int64),
            self.eos_token,
            torch.tensor([self.pad_token] * dec_num_padding_tokens, dtype=torch.int64),
        ],
        dim=0,
    )

    assert encoder_input.size(0) == self.seq_len
    assert decoder_input.size(0) == self.seq_len
    assert label.size(0) == self.seq_len

    return {
        "encoder_input": encoder_input,
        "decoder_input": decoder_input,
        "encoder_mask": (encoder_input != self.pad_token).unsqueeze(0).unsqueeze(0).int(),
        "decoder_mask": (decoder_input != self.pad_token).unsqueeze(0).int() & causal_mask(decoder_input.size(0)),
        "label": label,
        "src_text": src_text,
        "tgt_text": tgt_text,
    }

def causal_mask(size):
  mask = torch.triu(torch.ones((1, size, size)), diagonal=1).type(torch.int)
  return mask == 0


## Constriumos el Tokenizer


In [ ]:
!pip install \
  nvidia-cublas-cu12==12.4.5.8 \
  nvidia-cuda-cupti-cu12==12.4.127 \
  nvidia-cuda-nvrtc-cu12==12.4.127 \
  nvidia-cuda-runtime-cu12==12.4.127 \
  nvidia-cudnn-cu12==9.1.0.70 \
  nvidia-cufft-cu12==11.2.1.3 \
  nvidia-curand-cu12==10.3.5.147 \
  nvidia-cusolver-cu12==11.6.1.9 \
  nvidia-cusparse-cu12==12.3.1.170 \
  nvidia-nvjitlink-cu12==12.4.127

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 132.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 101.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 115.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvji

In [ ]:
!pip install transformers tokenizers datasets -qU

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [ ]:
def get_all_sentences(ds, lang):
    for item in ds:
      yield item['translation'][lang]

Entrenaremos rápidamente un tokenizador en nuestro conjunto de datos tanto para nuestras lenguas fuente como objetivo. Nos aseguraremos de añadir los tokens especiales [UNK], [PAD], [SOS] y [EOS].

Esto significa que se utilizará un proceso para enseñar al tokenizador a convertir el texto en secuencias numéricas, trabajando tanto con el idioma de origen como con el idioma al que se quiere traducir. Además, se incluirán cuatro tokens especiales con funciones específicas:

- [UNK]: Representa cualquier palabra que el tokenizador no reconozca ("unknown").
- [PAD]: Se utiliza para rellenar las secuencias hasta una longitud uniforme, facilitando el procesamiento por modelos que requieren entradas de tamaño fijo.
- [SOS]: Marca el inicio de una secuencia de texto ("start of sequence").
- [EOS]: Indica el final de una secuencia de texto ("end of sequence").

Añadir estos tokens especiales es crucial para el procesamiento y el entrenamiento eficaz de modelos de lenguaje, permitiendo al modelo reconocer y manejar situaciones como el comienzo y el final de las frases, así como gestionar entradas de diversas longitudes de manera más eficiente.


In [ ]:
from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace

def build_tokenizer(config, ds, lang):
  tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))
  tokenizer.pre_tokenizer = Whitespace()
  trainer = WordLevelTrainer(special_tokens=["[UNK]", "[PAD]", "[SOS]", "[EOS]"], min_frequency=2)
  tokenizer.train_from_iterator(get_all_sentences(ds, lang), trainer=trainer)
  return tokenizer

Creación del dataset

In [ ]:
from torch.utils.data import DataLoader, random_split

def get_ds(config):
  # Carga el conjunto de datos de la fuente especificada por la configuración, para un par de idiomas.
  ds_raw = load_dataset(f"{config['datasource']}", f"{config['lang_src']}-{config['lang_tgt']}", split='train')

  # Construye los tokenizadores para los idiomas de origen y destino basados en el conjunto de datos y la configuración.
  tokenizer_src = build_tokenizer(config, ds_raw, config['lang_src'])
  tokenizer_tgt = build_tokenizer(config, ds_raw, config['lang_tgt'])

  # Divide el conjunto de datos en un 90% para entrenamiento y un 10% para validación.
  train_ds_size = int(0.9 * len(ds_raw))
  val_ds_size = len(ds_raw) - train_ds_size
  train_ds_raw, val_ds_raw = random_split(ds_raw, [train_ds_size, val_ds_size])

  # Crea los conjuntos de datos bilingües de entrenamiento y validación con los tokenizadores y la máxima longitud de secuencia especificados.
  train_ds = BilingualDataset(train_ds_raw, tokenizer_src, tokenizer_tgt, config['lang_src'], config['lang_tgt'], config['seq_len'])
  val_ds = BilingualDataset(val_ds_raw, tokenizer_src, tokenizer_tgt, config['lang_src'], config['lang_tgt'], config['seq_len'])

  # Calcula la longitud máxima de las frases en los idiomas de origen y destino para determinar las dimensiones de relleno necesarias.
  max_len_src = 0
  max_len_tgt = 0

  for item in ds_raw:
    src_ids = tokenizer_src.encode(item['translation'][config['lang_src']]).ids
    tgt_ids = tokenizer_tgt.encode(item['translation'][config['lang_tgt']]).ids
    max_len_src = max(max_len_src, len(src_ids))
    max_len_tgt = max(max_len_tgt, len(tgt_ids))

  # Imprime la longitud máxima encontrada para las frases de origen y destino.
  print(f'Longitud máxima de la frase de origen: {max_len_src}')
  print(f'Longitud máxima de la frase de destino: {max_len_tgt}')

  # Crea cargadores de datos para los conjuntos de entrenamiento y validación, que permiten iterar sobre los conjuntos de datos en lotes durante el entrenamiento.
  train_dataloader = DataLoader(train_ds, batch_size=config['batch_size'], shuffle=True)
  val_dataloader = DataLoader(val_ds, batch_size=1, shuffle=True)

  # Retorna los cargadores de datos y los tokenizadores para ambos idiomas.
  return train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt


Necesitamos algunas funciones de ayuda

In [ ]:
def get_model(config, vocab_src_len, vocab_tgt_len):
  model = build_transformer(vocab_src_len, vocab_tgt_len, config["seq_len"], config['seq_len'], d_model=config['d_model'])
  return model

In [ ]:
def get_weights_file_path(config, epoch: str):
  model_folder = f"{config['datasource']}_{config['model_folder']}"
  model_filename = f"{config['model_basename']}{epoch}.pt"
  return str(Path('.') / model_folder / model_filename)

In [ ]:
def latest_weights_file_path(config):
  model_folder = f"{config['datasource']}_{config['model_folder']}"
  model_filename = f"{config['model_basename']}*"
  weights_files = list(Path(model_folder).glob(model_filename))
  if len(weights_files) == 0:
      return None
  weights_files.sort()
  return str(weights_files[-1])

Este código define el proceso de entrenamiento para un modelo basado en la arquitectura Transformer utilizando PyTorch. Aquí se detalla cada parte del código:

1. **Configuración del Entorno de Ejecución**: Inicialmente, determina si se puede utilizar CUDA (para GPUs NVIDIA), MPS (para aceleración en hardware Apple), o en su defecto, CPU para el entrenamiento. Esto optimiza el rendimiento de entrenamiento según el hardware disponible.

2. **Comprobación y Creación de Directorios**: Asegura que el directorio donde se guardarán los pesos del modelo existe, evitando errores al guardar los resultados de entrenamiento.

3. **Carga de Datos y Modelo**: Utiliza la función `get_ds` para obtener los cargadores de datos de entrenamiento y validación, junto con los tokenizadores para las lenguas de origen y destino. Después, carga el modelo y lo asigna al dispositivo de ejecución adecuado.

4. **Configuración de Optimizador y Función de Pérdida**: Se establece un optimizador Adam para ajustar los pesos del modelo durante el entrenamiento, junto con una función de pérdida de Cross Entropy que ignora los tokens de relleno (padding) y utiliza suavización de etiquetas para mejorar la generalización.

5. **Bucle de Entrenamiento**: Por cada época, itera sobre los lotes del conjunto de datos de entrenamiento. Para cada lote:

   - Se mueven los datos de entrada y etiquetas al dispositivo correcto.
   - Se ejecuta el proceso de codificación, decodificación y proyección dentro del modelo.
   - Se calcula la pérdida comparando la salida del modelo con las etiquetas reales.
   - Se actualizan los pesos del modelo utilizando la retropropagación (backpropagation) y el optimizador.

6. **Guardado de los Pesos del Modelo**: Después de cada época, se guardan los estados del modelo y del optimizador, así como el paso global, en un archivo. Esto permite reanudar el entrenamiento más adelante o utilizar el modelo entrenado para inferencia.

El código también incluye buenas prácticas como el uso de `torch.cuda.empty_cache()` para liberar memoria no utilizada en la GPU, así como la inicialización de variables para controlar la época inicial y el paso global de entrenamiento. Además, hace uso de `tqdm` para visualizar el progreso de entrenamiento en tiempo real.


In [ ]:
import warnings
from tqdm import tqdm
import os
from pathlib import Path

def train_model(config):
  # Definimos el dispositivo de ejecución
  device = "cuda" if torch.cuda.is_available() else "mps" if torch.has_mps or torch.backends.mps.is_available() else "cpu"
  print("Using device:", device)
  if (device == 'cuda'):
    print(f"Device name: {torch.cuda.get_device_name(device.index)}")
    print(f"Device memory: {torch.cuda.get_device_properties(device.index).total_memory / 1024 ** 3} GB")
  else:
    print("Please ensure you're in a GPU enabled Colab Notebook instance.")
  device = torch.device(device)

  # Asegúrate de que la carpeta de pesos existe
  Path(f"{config['datasource']}_{config['model_folder']}").mkdir(parents=True, exist_ok=True)

  train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt = get_ds(config)
  model = get_model(config, tokenizer_src.get_vocab_size(), tokenizer_tgt.get_vocab_size()).to(device)

  optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'], eps=1e-9)

  initial_epoch = 0
  global_step = 0

  loss_fn = nn.CrossEntropyLoss(ignore_index=tokenizer_src.token_to_id('[PAD]'), label_smoothing=0.1).to(device)

  for epoch in range(initial_epoch, config['num_epochs']):
    torch.cuda.empty_cache()
    model.train()
    batch_iterator = tqdm(train_dataloader, desc=f"Procesando Época {epoch:02d}")
    for batch in batch_iterator:
      encoder_input = batch['encoder_input'].to(device)
      decoder_input = batch['decoder_input'].to(device)
      encoder_mask = batch['encoder_mask'].to(device)
      decoder_mask = batch['decoder_mask'].to(device)

      encoder_output = model.encode(encoder_input, encoder_mask)
      decoder_output = model.decode(encoder_output, encoder_mask, decoder_input, decoder_mask)
      proj_output = model.project(decoder_output)

      label = batch['label'].to(device)

      loss = loss_fn(proj_output.view(-1, tokenizer_tgt.get_vocab_size()), label.view(-1))
      batch_iterator.set_postfix({"loss": f"{loss.item():6.3f}"})

      loss.backward()

      optimizer.step()
      optimizer.zero_grad(set_to_none=True)

      global_step += 1

    model_filename = get_weights_file_path(config, f"{epoch:02d}")
    torch.save({
      'epoch': epoch,
      'model_state_dict': model.state_dict(),
      'optimizer_state_dict': optimizer.state_dict(),
      'global_step': global_step
    }, model_filename)


Opus Books => [link](https://huggingface.co/datasets/opus_books)

In [ ]:
config = {
  "batch_size": 16,
  "num_epochs": 1, #modifica a 5 para mejores resultados
  "lr": 10**-4,
  "seq_len": 350,
  "d_model": 512,
  "datasource": 'opus_books',
  "lang_src": "en",
  "lang_tgt": "es",
  "model_folder": "weights",
  "model_basename": "encoder_decoder_model_"
}

Con una A100 tarda unos 20 minutos

In [ ]:
train_model(config)

Using device: cuda
Device name: NVIDIA A100-SXM4-40GB
Device memory: 39.55743408203125 GB
Longitud máxima de la frase de origen: 767
Longitud máxima de la frase de destino: 782


Procesando Época 00: 100%|██████████| 5258/5258 [18:02<00:00,  4.86it/s, loss=5.439]


In [ ]:
import torch

def load_model(config):
    # Carga el modelo y los tokenizadores
    _, _, tokenizer_src, tokenizer_tgt = get_ds(config)
    model = get_model(config, tokenizer_src.get_vocab_size(), tokenizer_tgt.get_vocab_size())

    # Cargar los pesos del modelo
    model_path = get_weights_file_path(config, "00")  # Asegúrate de tener un archivo final o especificar la época
    checkpoint = torch.load(model_path, map_location=config['device'])
    model.load_state_dict(checkpoint['model_state_dict'])

    return model, tokenizer_src, tokenizer_tgt

def translate(text, model, tokenizer_src, tokenizer_tgt, config):
    model.eval()  # Poner el modelo en modo de evaluación
    model.to(config['device'])

    # Preprocesar el texto de entrada
    tokens_src = tokenizer_src.encode(text)
    tokens_src_tensor = torch.tensor(tokens_src.ids).unsqueeze(0).to(config['device'])  # Correcto
    encoder_mask = torch.ones(tokens_src_tensor.shape).to(config['device'])

    with torch.no_grad():
        # Pasar por el modelo
        encoder_output = model.encode(tokens_src_tensor, encoder_mask)  # Utilizar tokens_src_tensor aquí
        # Iniciar con el token inicial para la decodificación
        decoder_input = torch.tensor([[tokenizer_tgt.token_to_id('[SOS]')]]).to(config['device'])
        output_tokens = []

        # Decodificación autoregresiva
        for _ in range(210):  # Definir un máximo de longitud de salida
            decoder_mask = torch.ones(decoder_input.shape).to(config['device'])
            decoder_output = model.decode(encoder_output, encoder_mask, decoder_input, decoder_mask)
            proj_output = model.project(decoder_output[:, -1, :])
            next_token_id = proj_output.argmax(1).item()
            if next_token_id == tokenizer_tgt.token_to_id('[EOS]'):
                break  # Finalizar si se encuentra el token de fin
            output_tokens.append(next_token_id)
            decoder_input = torch.cat([decoder_input, torch.tensor([[next_token_id]], device=config['device'])], dim=1)

        translated_text = tokenizer_tgt.decode(output_tokens)  # Decodificar los tokens de salida a texto

    return translated_text

# Configura el dispositivo correctamente
config['device'] = "cuda" if torch.cuda.is_available() else "mps" if torch.has_mps() else "cpu"

# Cargar el modelo y los tokenizadores
model, tokenizer_src, tokenizer_tgt = load_model(config)

# Ejemplo de uso
text = "Good Morning"
translated_text = translate(text, model, tokenizer_src, tokenizer_tgt, config)
print(f"Result: {translated_text}")

text = "Are you alive?"
translated_text = translate(text, model, tokenizer_src, tokenizer_tgt, config)
print(f"Result: {translated_text}")


Longitud máxima de la frase de origen: 767
Longitud máxima de la frase de destino: 782
Result: ¡ !
Result: ¿ qué ?
